In [1]:
import pandas as pd
import numpy as np
import sys 
import datetime
import os
import matplotlib as plt

%matplotlib inline

In [2]:
agg = {'Fare':'mean','Trip Total':'mean','Trip Miles':'sum','Trips Pooled':'sum','PRIVATE_TRIPS':'sum','SHARED_TRIPS':'sum','TRIPS':'sum'}

In [3]:
def clean_float_cols(x):
    x = x.replace(',','')
    x = float(x)
    
    return x

In [5]:
chunk = pd.read_csv('C:/Workspace/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Transportation_Network_Providers_-_Trips.csv', nrows = 100000, parse_dates = ['Trip Start Timestamp','Trip End Timestamp'], infer_datetime_format = True)

In [6]:
chunk.head()

,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,...,Additional Charges,Trip Total,Shared Trip Authorized,Trips Pooled,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,4d6dd683d524113ac3bb245074c7984d17ca67c3,2019-04-05 23:45:00,2019-04-06 00:00:00,336.0,2.1,1.703183e+10,1.703116e+10,4.0,16.0,5.0,...,2.55,7.55,False,1,41.965142,-87.676578,POINT (-87.6765780714 41.9651417087),41.959210,-87.712973,POINT (-87.7129731725 41.9592101847)
1,4d6dd78ccb72fe467a67ba7c50d78f8dca90c3bb,2019-05-10 11:15:00,2019-05-10 12:15:00,3342.0,33.1,1.703138e+10,NaN,38.0,NaN,45.0,...,2.55,47.55,False,1,41.813806,-87.623768,POINT (-87.6237680464 41.8138063365),NaN,NaN,NaN
2,4d6dd85fd7badd09d97af0c0b96581cdb7f0b907,2019-04-16 14:30:00,2019-04-16 14:30:00,341.0,0.8,1.703132e+10,1.703184e+10,32.0,32.0,5.0,...,2.55,9.55,False,1,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707)
3,4d6dd863260cf313a3f82c03efa010386bf842d8,2019-05-25 20:30:00,2019-05-25 20:30:00,336.0,2.0,NaN,NaN,9.0,10.0,5.0,...,2.55,7.55,False,1,42.007613,-87.813781,POINT (-87.8137810343 42.0076125931),41.985015,-87.804532,POINT (-87.8045320063 41.9850151008)
4,4d6dd8e9263f7818ca6db6c582c5ab2b0c627c17,2019-04-06 19:00:00,2019-04-06 19:00:00,402.0,1.4,1.703124e+10,1.703183e+10,24.0,28.0,5.0,...,2.55,7.55,False,1,41.892355,-87.674507,POINT (-87.674506788 41.8923550478),41.885281,-87.657233,POINT (-87.6572331997 41.8852813201)


In [10]:
chunk.columns


Index(['Trip ID', 'Trip Start Timestamp', 'Trip End Timestamp', 'Trip Seconds',
       'Trip Miles', 'Pickup Census Tract', 'Dropoff Census Tract',
       'Pickup Community Area', 'Dropoff Community Area', 'Fare', 'Tip',
       'Additional Charges', 'Trip Total', 'Shared Trip Authorized',
       'Trips Pooled', 'Pickup Centroid Latitude', 'Pickup Centroid Longitude',
       'Pickup Centroid Location', 'Dropoff Centroid Latitude',
       'Dropoff Centroid Longitude', 'Dropoff Centroid Location',
       'TRAVEL_TIME_MINUTES', 'TRIP_LENGTH_MILES', 'ORIGIN', 'DESTINATION',
       'ORIGIN_COM_AREA', 'DEST_COM_AREA', 'FARE', 'TIP', 'ADDITIONAL_CHARGES',
       'TOTAL_COST', 'SHARED_FLAGGER', 'NUM_TRIPS_POOLED', 'SPEED'],
      dtype='object')

In [8]:
#rename columns and drop unnecessary columns
chunk['TRAVEL_TIME_MINUTES'] = chunk['Trip Seconds']/60
chunk['TRIP_LENGTH_MILES'] = chunk['Trip Miles']
chunk['ORIGIN'] = chunk['Pickup Census Tract']
chunk['DESTINATION'] = chunk['Dropoff Census Tract']
chunk['ORIGIN_COM_AREA'] = chunk['Pickup Community Area']
chunk['DEST_COM_AREA'] = chunk['Dropoff Community Area']
chunk['FARE'] = chunk['Fare']
chunk['TIP'] = chunk['Tip']
chunk['ADDITIONAL_CHARGES'] = chunk['Additional Charges']
chunk['TOTAL_COST'] = chunk['Trip Total']
chunk['SHARED_FLAGGER'] = np.where(chunk['Shared Trip Authorized'], 1,0)
chunk['NUM_TRIPS_POOLED'] = chunk['Trips Pooled']
chunk['SPEED'] = chunk['TRIP_LENGTH_MILES']/(chunk['TRAVEL_TIME_MINUTES']/60)
chunk['TRIP_ID'] = chunk['Trip ID']
chunk['TRIP_START_TIME'] = chunk['Trip Start Timestamp']
chunk['TRIP_END_TIME'] = chunk['Trip End Timestamp']



In [11]:
chunk = chunk.drop(['Pickup Census Tract','Dropoff Census Tract','Trip Seconds','Trip Miles','Pickup Community Area','Dropoff Community Area','Fare','Tip','Additional Charges','Trip Total','Shared Trip Authorized','Trips Pooled','Trip ID','Trip Start Timestamp','Trip End Timestamp', 'Pickup Centroid Latitude', 'Pickup Centroid Longitude','Pickup Centroid Location', 'Dropoff Centroid Latitude', 'Dropoff Centroid Longitude', 'Dropoff Centroid Location',],axis = 1)


In [13]:
def assign_flaggers(row):
    if (~np.isnan(row['ORIGIN'])) & (~np.isnan(row['DESTINATION'])) & (~np.isnan(row['ORIGIN_COM_AREA'])) & (~np.isnan(row['DEST_COM_AREA'])):
        row['INTERNAL_UNSUPPRESSED_FLAGGER'] = 1
        row['INTERNAL_SUPPPRESSED_FLAGGER'] = 0
        row['EXTERNAL_FLAGGER'] = 0
        row['INTERNAL_EXTERNAL_FLAGGER'] = 0

    elif (np.isnan(row['ORIGIN'])) & (np.isnan(row['DESTINATION'])) & (~np.isnan(row['ORIGIN_COM_AREA'])) & (~np.isnan(row['DEST_COM_AREA'])):
        row['INTERNAL_UNSUPPRESSED_FLAGGER'] = 0
        row['INTERNAL_SUPPPRESSED_FLAGGER'] = 1
        row['EXTERNAL_FLAGGER'] = 0
        row['INTERNAL_EXTERNAL_FLAGGER'] = 0

    elif (~np.isnan(row['ORIGIN'])) & (np.isnan(row['DESTINATION'])) & (~np.isnan(row['ORIGIN_COM_AREA'])) & (~np.isnan(row['DEST_COM_AREA'])):
        print('Something went wrong!')

    elif (np.isnan(row['ORIGIN'])) & (~np.isnan(row['DESTINATION'])) & (~np.isnan(row['ORIGIN_COM_AREA'])) & (~np.isnan(row['DEST_COM_AREA'])):
        print('Something went wrong!')
        
    elif (np.isnan(row['ORIGIN_COM_AREA'])) & (~np.isnan(row['DEST_COM_AREA'])):
        row['INTERNAL_UNSUPPRESSED_FLAGGER'] = 0
        row['INTERNAL_SUPPPRESSED_FLAGGER'] = 0
        row['EXTERNAL_FLAGGER'] = 0
        row['INTERNAL_EXTERNAL_FLAGGER'] = 1

    elif (~np.isnan(row['ORIGIN_COM_AREA'])) & (np.isnan(row['DEST_COM_AREA'])):
        row['INTERNAL_UNSUPPRESSED_FLAGGER'] = 0
        row['INTERNAL_SUPPPRESSED_FLAGGER'] = 0
        row['EXTERNAL_FLAGGER'] = 0
        row['INTERNAL_EXTERNAL_FLAGGER'] = 1
        
    elif (np.isnan(row['ORIGIN_COM_AREA'])) & (np.isnan(row['DEST_COM_AREA'])):
        row['INTERNAL_UNSUPPRESSED_FLAGGER'] = 0
        row['INTERNAL_SUPPPRESSED_FLAGGER'] = 0
        row['EXTERNAL_FLAGGER'] = 1
        row['INTERNAL_EXTERNAL_FLAGGER'] = 0
        
    else:
        print('Something went wrong!')

    return row

In [14]:
chunk = chunk.apply(lambda row: assign_flaggers(row), axis = 1)

In [61]:
chunk.head()

,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,...,TIP,ADDITIONAL_CHARGES,TOTAL_COST,SHARED_FLAGGER,NUM_TRIPS_POOLED,SPEED,INTERNAL_UNSUPPRESSED_FLAGGER,INTERNAL_SUPPPRESSED_FLAGGER,EXTERNAL_FLAGGER,INTERNAL_EXTERNAL_FLAGGER
0,4d6dd683d524113ac3bb245074c7984d17ca67c3,2019-04-05 23:45:00,2019-04-06 00:00:00,336.0,2.1,1.703183e+10,1.703116e+10,4.0,16.0,5.0,...,0,2.55,7.55,0,1,22.500000,1,0,0,0
1,4d6dd78ccb72fe467a67ba7c50d78f8dca90c3bb,2019-05-10 11:15:00,2019-05-10 12:15:00,3342.0,33.1,1.703138e+10,NaN,38.0,NaN,45.0,...,0,2.55,47.55,0,1,35.655296,0,0,0,1
2,4d6dd85fd7badd09d97af0c0b96581cdb7f0b907,2019-04-16 14:30:00,2019-04-16 14:30:00,341.0,0.8,1.703132e+10,1.703184e+10,32.0,32.0,5.0,...,2,2.55,9.55,0,1,8.445748,1,0,0,0
3,4d6dd863260cf313a3f82c03efa010386bf842d8,2019-05-25 20:30:00,2019-05-25 20:30:00,336.0,2.0,NaN,NaN,9.0,10.0,5.0,...,0,2.55,7.55,0,1,21.428571,0,1,0,0
4,4d6dd8e9263f7818ca6db6c582c5ab2b0c627c17,2019-04-06 19:00:00,2019-04-06 19:00:00,402.0,1.4,1.703124e+10,1.703183e+10,24.0,28.0,5.0,...,0,2.55,7.55,0,1,12.537313,1,0,0,0


In [15]:
conflation = pd.read_csv('C:/Workspace/TNC-Demand-Model/Inputs/Chicago Community Areas/Community_Area_to_Census_Tract.csv', index_col =0)
conflation.head()

,area_num_1,GEOID
0,35,17031351500
0,35,17031351400
0,35,17031839600
0,35,17031351100
0,35,17031839500


In [16]:
def process_suppressed(row,conflation):

    if (np.isnan(row['ORIGIN'])) & (np.isnan(row['DESTINATION'])) & (~np.isnan(row['ORIGIN_COM_AREA'])) & (~np.isnan(row['DEST_COM_AREA'])):

        df = pd.DataFrame(data = [row.values] * len(row), columns = row.index)

        df = pd.merge(df[['ORIGIN_COM_AREA','DEST_COM_AREA']],conflation, how = 'left', left_on = 'ORIGIN_COM_AREA', right_on = 'area_num_1',left_index=True, right_index=True)
        df = pd.merge(df,conflation, how = 'left', left_on = 'DEST_COM_AREA', right_on = 'area_num_1', suffixes = ('_ORIGIN','_DEST'),left_index=True, right_index=True)


        df['SCALAR'] = np.random.dirichlet(np.ones(len(df)))
        df = df[df['SCALAR'] == df['SCALAR'].max()]
        
        row['ORIGIN'] = df['GEOID_ORIGIN'].values[0]
        row['DESTINATION'] = df['GEOID_DEST'].values[0]
        
    else:
        row['ORIGIN'] = row['ORIGIN']
        row['DESTINATION'] = row['DESTINATION']
        
    
    return row

In [157]:
#df = row.merge(conflation, how = 'left', left_on = 'ORIGIN_COM_AREA', right_on = ['area_num_1'])
#df = row.merge(df, how = 'left', left_on = 'DEST_COM_AREA', right_on = ['area_num_1'], suffixes = ('_ORIGIN','_DEST'))



In [17]:
df = pd.merge(chunk,conflation, how = 'left', left_on = 'ORIGIN_COM_AREA', right_on = 'area_num_1')

In [189]:
#df[['ORIGIN_COM_AREA','DEST_COM_AREA','GEOID','area_num_1']].head(25)

In [18]:
chunk = chunk.apply(lambda row: process_suppressed(row, conflation), axis = 1)

In [19]:
chunk[chunk['INTERNAL_SUPPPRESSED_FLAGGER'] == 0][['ORIGIN','DESTINATION','INTERNAL_EXTERNAL_FLAGGER','EXTERNAL_FLAGGER']]

,ORIGIN,DESTINATION,INTERNAL_EXTERNAL_FLAGGER,EXTERNAL_FLAGGER
0,1.703183e+10,1.703116e+10,0.0,0.0
1,1.703138e+10,NaN,1.0,0.0
2,1.703132e+10,1.703184e+10,0.0,0.0
4,1.703124e+10,1.703183e+10,0.0,0.0
6,1.703198e+10,1.703141e+10,0.0,0.0
...,...,...,...,...
99993,1.703184e+10,1.703107e+10,0.0,0.0
99994,1.703103e+10,1.703184e+10,0.0,0.0
99997,1.703107e+10,1.703184e+10,0.0,0.0
99998,1.703177e+10,NaN,0.0,1.0


In [20]:
chunk.columns

Index(['TRAVEL_TIME_MINUTES', 'TRIP_LENGTH_MILES', 'ORIGIN', 'DESTINATION',
       'ORIGIN_COM_AREA', 'DEST_COM_AREA', 'FARE', 'TIP', 'ADDITIONAL_CHARGES',
       'TOTAL_COST', 'SHARED_FLAGGER', 'NUM_TRIPS_POOLED', 'SPEED',
       'INTERNAL_UNSUPPRESSED_FLAGGER', 'INTERNAL_SUPPPRESSED_FLAGGER',
       'EXTERNAL_FLAGGER', 'INTERNAL_EXTERNAL_FLAGGER'],
      dtype='object')

In [21]:
store.close()

NameError: name 'store' is not defined